![](img/bitcoin_for_hackers.png)

This is a Jupyter notebook. We're going to use it to interact with a running Bitcoin node.

To run the code in a cell, press shift-enter.

In [ ]:
# Run some code
print("2 + 3 = {}".format(2 + 3))

In [ ]:
# Import our dependencies
%matplotlib inline

import hashlib
from pprint import pprint, pformat
from IPython import display

from ecc import PrivateKey, Keypair
from ledger import Ledger

# Digital Signatures

### Signing a message

Digital signing uses a *private* key and a *public* key:
- the *private* key is kept secret, and is used to *sign* a message.

In [ ]:
# Get a keypair
kp = Keypair()

print("Private key: {}\n".format(kp.privkey))
print("Public key: {}\n".format(kp.pubkey))

# Only someone with the private key can sign a message:
print("Message: 'Bitcoin for Hackers'\n")

message = 'Bitcoin for Hackers'
# convert message to an integer

sig = kp.privkey.sign(message)
print("Signature: {}\n".format(sig))

### Verifying a message

- the *public* key is shared publicly, and is used to *validate* a signed message.

In [ ]:
# use the public key to verify the signature
is_valid = kp.pubkey.verify(message, sig)
print("Signature {}".format("valid" if is_valid else "invalid"))

In [ ]:
# is the signature still valid if we change the message?
is_valid = kp.pubkey.verify('Bitcoin for Hackerz', sig)
print("Signature {}".format("valid" if is_valid else "invalid"))

In [ ]:
# is a different signature valid for the message?
sig.r += 1
is_valid = kp.pubkey.verify(message, sig)
print("Signature {}".format("valid" if is_valid else "invalid"))

# Coins and Transactions

Introducing Alice, Bob and Charlie. Alice starts with one coin. We're going to pass that coin around by signing and validating transactions.

First let's set everything up, and check the balances and transaction history.

In [ ]:
# Generate keypairs for Alice, Bob and Charlie and print them out
keypairs = {}
for user in ['Alice', 'Bob', 'Charlie']:
    keypairs[user] = Keypair()

# print the keypairs
pprint(keypairs)

users = sorted(keypairs.keys())
print(users)

In [ ]:
# Define what a 'Transaction' is and start a global ledger of transactions
class Transaction():
    def __init__(self, parent, recipient):
        self.parent = parent
        self.recipient = recipient
        if self.parent is None:
            # Special case for a 'genesis tranasction'
            p = b''
        else:
            p = self.parent
        self.id = hashlib.sha256(p + self.recipient.sec()).digest()
        self.signature = b''
        
    def __repr__(self):
        if self.parent is None:
            parent = 'None        '
        else:
            parent = '0x{:<7.7}...'.format(self.parent.hex())
        if self.signature == b'':
            sig = 'None            '
        else:
            sig = self.signature
        return "Txid: 0x{:<7.7}..., parent: {}, recipient: {:<8}, signature: {}".format(self.id.hex(), parent, self.recipient_name, sig)
    
    @property
    def recipient_name(self):
        for user in keypairs:
            if keypairs[user].pubkey == self.recipient:
                return user
    
    def sign(self, priv_key):
        "Sign the transaction id using a private key"
        self.signature = priv_key.sign(int.from_bytes(self.id, 'little'))
        
    def send(self):
        "Add a signed transaction to the global ledger"
        if self.verify:
            transactions.add_transaction(self)
        
    def verify(self):
        "Verify that the signature for the transaction is valid"
        if self.parent is None:
            # Special case for a 'genesis transaction'
            return True
        parent_tx = None
        for tx in transactions.get_transactions():
            if tx.id == self.parent:
                parent_tx = tx
        if parent_tx is None:
            return False  # parent transaction not found
        return parent_tx.recipient.verify(int.from_bytes(self.id, 'little'), self.signature)

transactions = Ledger(users)

In [ ]:
# Start by adding a 'genesis' transaction for Alice

tx = Transaction(None, keypairs['Alice'].pubkey)
tx.send()

# print out transaction history
pprint(transactions)

In [ ]:
# print and draw balances
print(transactions.get_balances())
transactions.draw_balances()

### Sending a transaction

Only Alice knows her private key, so only she can sign a transaction that sends her coin.

Alice can send her coin to Bob.

In [ ]:
tx2 = Transaction(tx.id, keypairs['Bob'].pubkey)
tx2.sign(keypairs['Alice'].privkey)
tx2.send()

Anyone can verify the transaction with Alice's *public* key

In [ ]:
is_valid = tx2.verify()
print("Transaction {:<7.7}... is {}".format(tx.id.hex(), "valid" if is_valid else "invalid"))

In [ ]:
# print out transaction history
pprint(transactions)

In [ ]:
# print and draw balances
print(transactions.get_balances())
transactions.draw_balances()

Bob can now sign a new transaction, sending his coin to Charlie.

In [ ]:
tx3 = Transaction(tx2.id, keypairs['Charlie'].pubkey)
tx3.sign(keypairs['Bob'].privkey)
tx3.send()

In [ ]:
# print out transaction history
pprint(transactions)

In [ ]:
# print and draw balances
print(transactions.get_balances())
transactions.draw_balances()

# The Double spend problem

Bob can sign a second transaction, spending the same coins!

In [ ]:
tx3b = Transaction(tx2.id, keypairs['Alice'].pubkey)
tx3b.sign(keypairs['Bob'].privkey)
tx3b.send()

In [ ]:
# print out transaction history
pprint(transactions)

In [ ]:
# print and draw balances
print(transactions.get_balances())
transactions.draw_balances()

### Ordering

Let's change our ledger logic so only the first spend of a coin is valid.

In [ ]:
# print out transaction history, showing spentness validity
print(transactions.list_with_spentness())

In [ ]:
# print and draw balances, disallowing doublespends
print(transactions.get_balances(allow_doublespends=False))
transactions.draw_balances(allow_doublespends=False)

Seems to have fixed things, right?

Not so fast!

On a distributed network, there isn't a *canonical* ordering. What if Bob sends the Alice transaction to Alice, and the Charlie transaction to Charlie. Alice sees her transaction first, so her view of the ledger is as above. Charlie sees his transaction first, so he sees:

In [ ]:
transactions.swap_order(-1, -2)
print(transactions.list_with_spentness())

In [ ]:
# print and draw balances, disallowing doublespends
print(transactions.get_balances(allow_doublespends=False))
transactions.draw_balances(allow_doublespends=False)

![](img/ruh_roh.jpg)

# Cryptographic hashes

Cryptographic hashes have the following properties:

1. it is *infeasible* to generate a message from its hash value (preimage resistance)
2. a small change to a message results in a completely different digest (avalanche effect)
3. it is *infeasible* to find two different messages with the same hash value (collision resistance)

Let's test (1) and (2):

- What's the double sha256 digest of "Bitcoin for Hackers"?
- What's the double sha256 digest of "Bitcoin for Hackerz"?
- What's the preimage for the digest "0x707e397fc6c0327b6d9c3a2be68d4fd2456609caa53f8a4fdc7131d3944b516c"?

In [ ]:
def hash(message):
    # hash the block with the nonce
    print("{0:#0{1}x}".format(int.from_bytes(helper.double_sha256((message).encode()), 'big'), 66))

hash(message="Bitcoin for Hackers")

# Proof-of-Work

Proof-of-work determines who gets to build the next block in the Bitcoin blockchain. It relies on the fact that a cryptographic hash function is one-way, and the output is essentially randomly distributed.

We're going to do some manual proof-of-work over a short message.

Find a valid 'block' for the message "Bitcoin for Hackers" with 4 bits of difficulty.

Try running the `validate_block()` function with the same message but different nonces.

In [ ]:
def validate_block(message, nonce, difficulty):
    # hash the block with the nonce
    block_hash = int.from_bytes(helper.double_sha256((message + nonce).encode()), 'big')
    if block_hash < 2 ** (256 - difficulty):
        # winner! Our digest was below the target difficulty
        print("Valid block:    {0:#0{1}x}.\nNonce = {2}".format(block_hash, 66, nonce))
        return True
    else:
        # Sorry, your digest was too large!
        print("Invalid block:  {0:#0{1}x}.\nNonce = {2}".format(block_hash, 66, nonce))
        return False

validate_block(message="Bitcoin for Hackers", nonce="nonce1", difficulty=4)

# Validating work

Proof-of-work is *hard* to do, but *easy* to validate. Once you and your neighbor have a valid block, verify that their block is valid by running `validate_block()` with their nonce.

In [ ]:
validate_block(message="Bitcoin for Hackers", nonce=<put a valid nonce here>, difficulty=4)